# create bucket

In [2]:
import boto3

In [3]:
with open('secret.csv') as f:
    ACCESS_KEY = f.readline().strip()
    SECRET_KEY = f.readline().strip()

In [4]:
s3 = boto3.resource('s3', 
    aws_access_key_id=ACCESS_KEY, 
    aws_secret_access_key=SECRET_KEY, 
) 

In [6]:
try: 
    s3.create_bucket(Bucket='shoot-buckets', CreateBucketConfiguration={ 
        'LocationConstraint': 'us-west-2'}) 
except Exception as e: 
    print (e) 

In [7]:
bucket = s3.Bucket("shoot-buckets")
bucket 

s3.Bucket(name='shoot-buckets')

In [8]:
bucket.Acl().put(ACL='public-read') 

{'ResponseMetadata': {'RequestId': '1EB322G2JXP9NPY1',
  'HostId': 'a56UzdcEVjhRtpRy0QPrCu2t3TLCA3Pn0bbCGbVw7BTBqUT2Ox4uE+1wGnLxWYVVbwRRlWtUbCs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'a56UzdcEVjhRtpRy0QPrCu2t3TLCA3Pn0bbCGbVw7BTBqUT2Ox4uE+1wGnLxWYVVbwRRlWtUbCs=',
   'x-amz-request-id': '1EB322G2JXP9NPY1',
   'date': 'Mon, 25 Oct 2021 21:50:22 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1}}

In [12]:
with open('exp1.csv', 'rb') as body:
    o = s3.Object('shoot-buckets', 'test').put(Body=body)

In [14]:
s3.Object('shoot-buckets', 'test').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'FMT7VAZMQP4J2WVJ',
  'HostId': 'ay/JjhLkHv5Y/kNJuMqMIsqGHEHu7I4GDOUbWPS5YaulvlBrH8MYwAUVkuCdX2t79yrfa8pQCp8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ay/JjhLkHv5Y/kNJuMqMIsqGHEHu7I4GDOUbWPS5YaulvlBrH8MYwAUVkuCdX2t79yrfa8pQCp8=',
   'x-amz-request-id': 'FMT7VAZMQP4J2WVJ',
   'date': 'Mon, 25 Oct 2021 21:51:56 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [19]:
dyndb = boto3.resource('dynamodb', 
    region_name='us-west-2', 
    aws_access_key_id=ACCESS_KEY, 
    aws_secret_access_key=SECRET_KEY,
) 
dyndb

dynamodb.ServiceResource()

In [20]:
try: 
    table = dyndb.create_table( 
        TableName='DataTable', 
        KeySchema=[ 
            { 
                'AttributeName': 'PartitionKey', 
                'KeyType': 'HASH' 
            }, 
            { 
                'AttributeName': 'RowKey', 
                'KeyType': 'RANGE' 
            } 
        ], 
        AttributeDefinitions=[ 
            { 
                'AttributeName': 'PartitionKey', 
                'AttributeType': 'S' 
            }, 
            { 
                'AttributeName': 'RowKey', 
                'AttributeType': 'S' 
            }, 
 
        ], 
        ProvisionedThroughput={ 
            'ReadCapacityUnits': 5, 
            'WriteCapacityUnits': 5 
        } 
    ) 
except Exception as e: 
    print (e) 
    #if there is an exception, the table may already exist.   if so... 
    table = dyndb.Table("DataTable")

In [21]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [22]:
print(table.item_count) 

0


# upload data

In [26]:
import csv

In [36]:
with open('experiments.csv', 'r') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for _ in csvf: break
    for item in csvf:
        print(item)
        with open(item[4], 'rb') as body:
            s3.Object('shoot-buckets', item[4]).put(Body=body)
            md = s3.Object('shoot-buckets', item[4]).Acl().put(ACL='public-read')

            url = " https://s3-us-west-2.amazonaws.com/shoot-buckets/"+item[4]
            metadata_item = {'PartitionKey': item[4], 'RowKey': item[0], 'Temp': item[1],
                            'Conductivity': item[2], 'Concentration': item[3], 'url': url}
            try:
                table.put_item(Item=metadata_item)
            except Exception as e:
                raise e
                print("item may already be there or another failure")


['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [40]:
response = table.get_item( 
    Key={ 
        'PartitionKey': 'exp3.csv', 
        'RowKey': '3' 
    } 
) 
# print(response)
item = response['Item'] 
print(item) 

{'Item': {'Temp': '-2.93', 'RowKey': '3', 'Conductivity': '57.1', 'Concentration': '3.7', 'PartitionKey': 'exp3.csv', 'url': ' https://s3-us-west-2.amazonaws.com/shoot-buckets/exp3.csv'}, 'ResponseMetadata': {'RequestId': '1I6SDLJIN22TBLUTAS6ICROBHNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 25 Oct 2021 22:16:13 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '211', 'connection': 'keep-alive', 'x-amzn-requestid': '1I6SDLJIN22TBLUTAS6ICROBHNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '482430803'}, 'RetryAttempts': 0}}
{'Temp': '-2.93', 'RowKey': '3', 'Conductivity': '57.1', 'Concentration': '3.7', 'PartitionKey': 'exp3.csv', 'url': ' https://s3-us-west-2.amazonaws.com/shoot-buckets/exp3.csv'}


In [41]:
response

{'Item': {'Temp': '-2.93',
  'RowKey': '3',
  'Conductivity': '57.1',
  'Concentration': '3.7',
  'PartitionKey': 'exp3.csv',
  'url': ' https://s3-us-west-2.amazonaws.com/shoot-buckets/exp3.csv'},
 'ResponseMetadata': {'RequestId': '1I6SDLJIN22TBLUTAS6ICROBHNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 25 Oct 2021 22:16:13 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '211',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1I6SDLJIN22TBLUTAS6ICROBHNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '482430803'},
  'RetryAttempts': 0}}